In [ ]:
import pickle as pkl
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
import os 

#raw_data_all_path = '/Users/ziegler/Documents/data/202142_raw_data_all_prml.pkl'

#with open(raw_data_all_path, 'rb') as infile:
#    raw_data_all = pkl.load(infile)

In [ ]:
names = raw_data_all['sims']
x = raw_data_all['x']

In [ ]:
# define noise variance
temp = 10
bandwidth = 200e6
kB = 1.38e-23
R = 50

noise_var = kB * temp * bandwidth * R

#random_data = rng.multivariate_normal([0., 0.], np.eye(2) * noise_var / 2, N)

# pitch angles
pitch_angles = []
for name in names:
    pitch_angles.append(float(name.split('Angle')[-1].split('_Pos')[0]))

pitch_angles = np.array(pitch_angles)
train_pitch_angle = np.arange(87., 90.005, 0.005)
test_pitch_angle = np.arange(87.0025, 90., 0.005)


In [ ]:
# random number generation
seed_random = 666
N = 8192
rng = np.random.default_rng(seed = seed_random)

# noisy test, train, validation, data

unsorted_train_pitch_angle = []
train_data = []

unsorted_test_pitch_angle = []
test_data = []

unsorted_val_pitch_angle = []
val_data = []

n_train = 100 # number of train example per pitch angle.
n_test = 10 # number of test example per pitch angle.

for i, pa in enumerate(pitch_angles):
    
    if len(np.argwhere(abs(train_pitch_angle - pa) < 1e-3)) > 0:
        for k in range(n_train):
            unsorted_train_pitch_angle.append(pa)
            noise = rng.normal(loc = 0, scale = np.sqrt(noise_var), size = N)
            noisy_x = noise + np.real(x[i, :])
            noisy_x = 2 * noisy_x / (np.max(noisy_x) - np.min(noisy_x))
            
            train_data.append(np.float32(noisy_x))
            
    elif len(np.argwhere(abs(test_pitch_angle - pa) < 1e-3)) > 0:
        for k in range(n_test):
            unsorted_test_pitch_angle.append(pa)
            unsorted_val_pitch_angle.append(pa)
            
            noise = rng.normal(loc = 0, scale = np.sqrt(noise_var), size = N)
            noisy_x = noise + np.real(x[i, :])
            noisy_x = 2 * noisy_x / (np.max(noisy_x) - np.min(noisy_x))
            
            test_data.append(np.float32(noisy_x))
            
            noise = rng.normal(loc = 0, scale = np.sqrt(noise_var), size = N)
            noisy_x = noise + np.real(x[i, :])
            noisy_x = 2 * noisy_x / (np.max(noisy_x) - np.min(noisy_x))
            
            val_data.append(np.float32(noisy_x))
            
    if i % 100 == 99:
        print('Done with %.2d' % (i + 1))
        
# noise only data

train_noise = []
test_noise = []
val_noise = []

for i in range(len(unsorted_train_pitch_angle)):
    noise = rng.normal(loc = 0, scale = np.sqrt(noise_var), size = N)
    noise = 2 * noise / (np.max(noise) - np.min(noise))
    train_data.append(noise)
    
for i in range(len(unsorted_test_pitch_angle)):
    noise = rng.normal(loc = 0, scale = np.sqrt(noise_var), size = N)
    noise = 2 * noise / (np.max(noise) - np.min(noise))
    test_data.append(noise)
    
    noise = rng.normal(loc = 0, scale = np.sqrt(noise_var), size = N)
    noise = 2 * noise / (np.max(noise) - np.min(noise))
    val_data.append(noise)


        


In [ ]:
# save data
output = '/Users/ziegler/repos/romulus/output'

with open(os.path.join(output, 'train_data_prml.pkl'), 'wb') as outfile:
    pkl.dump(train_data, outfile)
    
with open(os.path.join(output, 'test_data_prml.pkl'), 'wb') as outfile:
    pkl.dump(test_data, outfile)
    
with open(os.path.join(output, 'val_data_prml.pkl'), 'wb') as outfile:
    pkl.dump(val_data, outfile)
    
with open(os.path.join(output, 'train_pitch_angles_prml.pkl'), 'wb') as outfile:
    pkl.dump(unsorted_train_pitch_angle, outfile)
    
with open(os.path.join(output, 'test_pitch_angles_prml.pkl'), 'wb') as outfile:
    pkl.dump(unsorted_test_pitch_angle, outfile)
    
with open(os.path.join(output, 'val_pitch_angles_prml.pkl'), 'wb') as outfile:
    pkl.dump(unsorted_val_pitch_angle, outfile)
    
train_labels = np.concatenate((np.ones(len(train_data) // 2), np.zeros(len(train_data) // 2)))
test_labels = np.concatenate((np.ones(len(test_data) // 2), np.zeros(len(test_data) // 2)))

with open(os.path.join(output, 'train_labels_prml.pkl'), 'wb') as outfile:
    pkl.dump(train_labels, outfile)
    
with open(os.path.join(output, 'test_labels.pkl'), 'wb') as outfile:
    pkl.dump(test_labels, outfile)
    
print(test_labels)

In [ ]:
test_data